## Interferometry for dummies notebook

This notebook should take you through some of the concepts that have been discussed in the lecture. Therefore, clues to all the solutions should be present in the slides. The purpose here is to get some hands-on experience, not to get hung up on coding issues. If you are stuck, there is an equivalent notebook containing the solutions from which you can draw inspiration.

In [ ]:
import numpy as np
from copy import copy

import matplotlib.pyplot as plt
from astropy.convolution import convolve, Gaussian2DKernel
from astropy.io import ascii, fits

### Fourier transforms in 1D

To get a feel for the fast Fourier transform functionality in numpy, see if you can get the primary beam pattern of single dish, assuming the aperture is uniformly illuminated.

As the Fourier transform is discrete and finite, the resulting function will not be perfect. Check to see how the size of the aperture relative to the sampled space and the sampling interval influence the result.

**If your results are strange or unexpected**: Whereas we like our signals to be in the middle of the sampled space for aesthetic reasons, the discrete Fourier transform follows the python indexing, which means that the 0-point will be right at the edges of the array. In general, `np.fft.fftshift` is your best friend here, and it is best to use it before and after every Fourier transform. Trust me, this will save you a lot of trouble.

In [ ]:
# Define amount of samples and extent of parameter space
nsamples=1024
X=100

x = np.linspace(-X/2,X/2,nsamples)
y = np.zeros(nsamples)

# Make tophat
y[np.logical_and(x>-0.5,x<0.5)] = 1
plt.plot(x,y)
plt.show()

# Fourier transform
y_fft = np.fft.fftshift(np.fft.fft(np.fft.fftshift(y)))
plt.plot(x, np.real(y_fft)*(X/nsamples))
plt.plot(x, np.sinc(x*(X/nsamples)))
plt.show()

### Simulating MeerKAT observations

In the data subdirectory, a text file is included with the UV-coverage of a MeerKAT observation. 

1. Start with plotting the UV-coverage, preferably in units of $k\lambda$, assuming $\lambda=$21 cm.

2. Using 2D Fourier transforms, convert the UV-coverage to a Point-spread-function (PSF). To do this you need to define a grid for UV space first, which will define the resolution and size of your resulting image. **Bonus**: Implement both natural and uniform weighting schemes, and see how this influences the shape of the PSF. 

3. In the data subdirectory there is a fits file containing the sky emission from a FRII radio galaxy. Using the data just obtained, convolve this image with PSF using 2D Fourier transforms.

In [ ]:
#Load in MeerKAT UV-coverage
meerkat_uv = ascii.read('data/meerkat_uv.txt')

In [ ]:
# Plot UV plane, and remember that each baseline has its conjugate!
plt.scatter(meerkat_uv['u']/0.21/1000,meerkat_uv['v']/0.21/1000, color='k', s=1)
plt.scatter(-meerkat_uv['u']/0.21/1000,-meerkat_uv['v']/0.21/1000, color='k', s=1)
plt.xlabel('u (k$\lambda$)')
plt.ylabel('v (k$\lambda$)')
plt.show()

In [ ]:
def uv_points_to_psf(u, v, imsize, weighting):
    '''
    Grid and Fourier transform UV coverage to create a PSF
    '''
    uv_image = np.zeros(shape=(imsize,imsize))

    if weighting == 'natural':
        for i in range(len(u)):
            uv_image[int(u[i]),int(v[i])] += 1
    if weighting == 'uniform':
        for i in range(len(u)):
            uv_image[int(u[i]),int(v[i])] = 1

    psf = np.fft.fftshift(np.fft.fft2(uv_image))

    return uv_image, psf

In [ ]:
# Get gridded UV coverage and PSF
uv_image, psf = uv_points_to_psf(meerkat_uv['u']/10, meerkat_uv['v']/10, imsize=4096, weighting='uniform')

plt.imshow(psf.real, cmap='YlGnBu_r', origin='lower')
plt.xlim(1948,2148)
plt.ylim(1948,2148)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# Load in radio galaxy fits file
radio_galaxy = fits.open('data/radio_galaxy.fits')[0].data

plt.imshow(radio_galaxy, cmap='YlGnBu_r')
plt.xlim(1548,2548)
plt.ylim(1548,2548)
plt.show()

# Convolve image with MeerKAT PSF
radio_sources_vis = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(radio_galaxy)))
convolved_vis = radio_sources_vis*np.fft.fftshift(uv_image)
convolved = np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(convolved_vis)))

plt.imshow(convolved.real, cmap='YlGnBu_r')
plt.xlim(1548,2548)
plt.ylim(1548,2548)
plt.show()

### Implementing CLEAN

In the data subdirectory you will find a dirty image containing a number of point sources. In the same directory an image of the PSF is included. 

To clean the image, implement the Högbom CLEAN algorithm as specified in the slides, including a gain factor, an iteration limit, and a noise treshold. To create the clean image, the 2D Gaussian fit to the central lobe of the beam is 7.8x5.6 pixels with PA of 75, I have already defined as an astropy kernel. I have also included a function which takes care of subtracting two images which are shifted w.r.t. each other.

In [ ]:
# Conversion factor from FWHM to std
a = 2*np.sqrt(2*np.log(2))

maj_std = 7.8/a
min_std = 5.6/a
theta = np.deg2rad(75)

# Define clean beam
clean_beam = Gaussian2DKernel(x_stddev = min_std,
                              y_stddev = maj_std,
                              theta=theta,
                              x_size=51, y_size=51)
# Normalize the beam
clean_beam_normed = clean_beam.array/clean_beam.array.max()

In [ ]:
def subtract_shifted_images(im1, im2, X, Y, amp):
    '''
    Subtract amp*im2 from im1, given an offset in X and Y
    '''
    im1_xshape = int(im1.shape[0]/2)
    im1_yshape = int(im1.shape[1]/2)
        
    im2_xshape = int(im2.shape[0])
    im2_yshape = int(im2.shape[1])
    
    minY = min(int(Y)-im1_yshape, 0)
    minX = min(int(X)-im1_xshape, 0)
    maxY = max(int(Y)+im1_yshape-im2_yshape, 0)
    maxX = max(int(X)+im1_xshape-im2_xshape, 0)

    im1[int(X)-im1_xshape-minX:int(X)+im1_xshape-maxX,
        int(Y)-im1_yshape-minY:int(Y)+im1_yshape-maxY] -= amp*im2[-minX:im2_xshape-maxX,-minY:im2_yshape-maxY]
    
    return im1

In [ ]:
def hogbom_clean(dirty_image, psf, gain, niter, thresh):
    '''
    Implement Hogbom clean with gain, niter and threshold
    '''
    model_image=np.zeros(dirty_image.shape)
    residual=copy(dirty_image)
    
    for i in range(niter):
        x, y = np.unravel_index(residual.argmax(), residual.shape)
        print(x,y,residual[x,y])
        
        amp = gain*residual[x,y]
        
        model_image[x,y] += amp
        residual = subtract_shifted_images(residual, psf, x, y, amp)
        
        if residual.max() < thresh:
            print(i)
            break

    return model_image, residual

In [ ]:
# Load in dirty image and PSF
dirty_image = fits.open('data/dirty_image.fits')
psf = fits.open('data/psf.fits')

# Run CLEAN
model_image, residual = hogbom_clean(dirty_image[0].data, psf[0].data, gain=0.1, niter=1000, thresh=5e-2)
# Create CLEAN image
clean_image = convolve(model_image, clean_beam_normed, normalize_kernel=False, nan_treatment='fill') + residual

In [ ]:
# Write model image to fits
hdu = fits.PrimaryHDU(model_image, header=dirty_image[0].header)
hdu.writeto('data/model_image.fits', overwrite=True)

#Write CLEAN image to fits
hdu = fits.PrimaryHDU(clean_image, header=dirty_image[0].header)
hdu.writeto('data/clean_image.fits', overwrite=True)